In [64]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os,fnmatch
import matplotlib as mpl
import matplotlib.cm as cm
import datetime

In [65]:
activities_filepath = "activities.csv"
activities_df = pd.read_csv(activities_filepath)

In [66]:
activities_df.columns

Index(['Activity ID', 'Activity Date', 'Activity Name', 'Activity Type',
       'Activity Description', 'Elapsed Time', 'Distance', 'Relative Effort',
       'Commute', 'Activity Gear', 'Filename', 'Athlete Weight', 'Bike Weight',
       'Elapsed Time.1', 'Moving Time', 'Distance.1', 'Max Speed',
       'Average Speed', 'Elevation Gain', 'Elevation Loss', 'Elevation Low',
       'Elevation High', 'Max Grade', 'Average Grade',
       'Average Positive Grade', 'Average Negative Grade', 'Max Cadence',
       'Average Cadence', 'Max Heart Rate', 'Average Heart Rate', 'Max Watts',
       'Average Watts', 'Calories', 'Max Temperature', 'Average Temperature',
       'Relative Effort.1', 'Total Work', 'Number of Runs', 'Uphill Time',
       'Downhill Time', 'Other Time', 'Perceived Exertion',
       'translation missing: en-US.lib.export.portability_exporter.activities.horton_values.type',
       'translation missing: en-US.lib.export.portability_exporter.activities.horton_values.start_time',


In [67]:
activity_type = 'ride'
chosen_columns = ['Activity Date', 'Activity Type', 'Elapsed Time', 'Distance', 'Relative Effort', 'Activity Gear', 'Filename', 'Moving Time', 'Elevation Gain', 'Max Grade', 'Average Heart Rate', 'Max Watts', 'Max Heart Rate', 'Calories', 'Weighted Average Power']
raw_data = activities_df[chosen_columns]
raw_data.head()

Activity Date Activity Type  Elapsed Time Distance  \
0  Aug 5, 2016, 12:01:30 AM          Ride          1657     6.19   
1  Aug 5, 2016, 11:41:05 PM          Ride          1339     5.07   
2   Aug 7, 2016, 1:03:46 PM          Ride          1689     4.47   
3   Aug 3, 2016, 4:00:53 PM           Run           720     1.61   
4   Aug 4, 2016, 4:00:54 PM           Run           705     1.61   

   Relative Effort Activity Gear                  Filename  Moving Time  \
0              NaN           NaN  activities/664886326.gpx       1597.0   
1              NaN           NaN  activities/665868043.gpx       1186.0   
2              NaN           NaN  activities/667708161.gpx       1254.0   
3              NaN           NaN                       NaN        720.0   
4              NaN           NaN                       NaN        705.0   

   Elevation Gain  Max Grade  Average Heart Rate  Max Watts  Max Heart Rate  \
0       17.002199        3.8                 NaN        NaN             NaN   
1       17.352900        4.1                 NaN        NaN             NaN   
2       13.939700        4.0                 NaN        NaN             NaN   
3             NaN        NaN                 NaN        NaN             NaN   
4             NaN        NaN                 NaN        NaN             NaN   

     Calories  Weighted Average Power  
0   69.141052                     NaN  
1   56.649204                     NaN  
2   44.400299                     NaN  
3  138.415466                     NaN  
4  138.415466                     NaN

In [68]:
raw_data["Elapsed Time (min)"] = raw_data["Elapsed Time"].apply(lambda min: min // 60)
raw_data["Moving Time (min)"] = raw_data["Moving Time"].apply(lambda min: min // 60)
raw_data = raw_data.drop(["Elapsed Time", "Moving Time"], axis=1)

/home/felix/anaconda3/envs/deep-learning/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/felix/anaconda3/envs/deep-learning/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
# create an indicator column for real calories from a power source and estimated calories 
powermeter_acquisition_date = datetime.date(2020, 9, 23)
trSeason1 = [datetime.date(2019, 9, 27), datetime.date(2020, 4, 5)]
trSeason2 = [datetime.date(2020, 9, 23), datetime.date.today()]

In [63]:

raw_data['Datetime'] = pd.to_datetime(raw_data['Activity Date']).dt.floor('h')
raw_data["Power Based Calories"] = np.where(
                                            ((raw_data['Activity Type'] == 'Ride') &
                                            ((raw_data['Activity Gear'] == '2018 Norco Threshold (CX)')) &
                                            (raw_data['Datetime'] >= powermeter_acquisition_date)), 
                                            1, 0)
raw_data.tail()

/home/felix/anaconda3/envs/deep-learning/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


Activity Date Activity Type Distance  Relative Effort  \
1070   Nov 24, 2020, 5:00:29 PM          Yoga        0              NaN   
1071   Nov 25, 2020, 2:15:56 AM          Ride        0              NaN   
1072  Nov 26, 2020, 12:00:25 AM          Ride    44.51             92.0   
1073   Nov 26, 2020, 7:07:58 PM          Ride    58.94            113.0   
1074   Nov 27, 2020, 5:10:42 PM          Ride    45.13             51.0   

                  Activity Gear                      Filename  Elevation Gain  \
1070                        NaN                           NaN             NaN   
1071  2018 Norco Threshold (CX)                           NaN             NaN   
1072  2018 Norco Threshold (CX)  activities/4693590105.tcx.gz             0.0   
1073  2018 Norco Threshold (CX)  activities/4697769856.tcx.gz             0.0   
1074  2018 Norco Threshold (CX)  activities/4701307133.tcx.gz             0.0   

      Max Grade  Average Heart Rate  Max Watts  Max Heart Rate  Calories  \
1070        NaN                 NaN        NaN             NaN       NaN   
1071        NaN                 NaN        NaN             NaN       NaN   
1072        0.0          162.477219        NaN             NaN    1099.0   
1073        0.0          161.553192        NaN             NaN    1388.0   
1074        0.0          153.779449        NaN             NaN    1036.0   

      Weighted Average Power  Elapsed Time (min)  Moving Time (min)  \
1070                     NaN                  40               40.0   
1071                     NaN                  20               20.0   
1072                   228.0                  90               90.0   
1073                   218.0                 120              120.0   
1074                   195.0                  90               90.0   

                Datetime  Power Based Calories  
1070 2020-11-24 17:00:00                     0  
1071 2020-11-25 02:00:00                     1  
1072 2020-11-26 00:00:00                     1  
1073 2020-11-26 19:00:00                     1  
1074 2020-11-27 17:00:00                     1